In [1]:
# IMPORTS
##########################

import agent
import environment
import doubledqn
import tools
import memory
import simulation
import multiprocessing
import pandas as pd
import csv
import os

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

import numpy as np
import matplotlib.pyplot as plt
import time
import itertools
from keras import optimizers 

def iter_params(**kwargs):
    keys = kwargs.keys()
    vals = kwargs.values()
    for instance in itertools.product(*vals):
        yield dict(zip(keys, instance))

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
# ONE RUN
#############################################

param = { 
    
    "batch_size" : 100, # 30,50,60,[70]
    "target_update_freq" : 10000,#10000], # 10000,20000,30000,[40000]
    "gamma" : 0.99,# # 0.98,0.99,0.995,[0.999]
    "train_freq" : 1, # 2,3,4,[5]
    "max_size" : 100000,#,70000], # 20000,50000,70000,[100000]
    "max_ep_length" : 2000, # 1000,2000,3000,[4000]
    "policy" : "epsGreedy",#, "epsGreedy"],
    "eps" : 0.1,#, 0.3, 0.1],
    "delta_time" : 10,
    #"optimizer": [optimizers.RMSprop(lr= 0.001), optimizers.Adagrad(), optimizers.Adam()]
}

sumo_RL = simulation.simulator(
                    connection_label = "single_worker",
                    q_network_type = 'simple',
                    target_q_network_type = 'simple',
                    gamma = param["gamma"],
                    target_update_freq = param["target_update_freq"],
                    train_freq = param["train_freq"],
                    num_burn_in = 200,
                    batch_size = param["batch_size"],
                    optimizer = optimizers.RMSprop(lr= 0.001),
                    loss_func = "mse",
                    max_ep_length = param["max_ep_length"],
                    experiment_id = "Testing_Reward",
                    model_checkpoint = True,
                    opt_metric = None,

                   # environment parameters
                    net_file = "cross.net.xml",
                    route_file = "cross.rou.xml",
                    demand = "nominal",
                    state_shape = (1,19),
                    num_actions = 2,
                    use_gui = True,
                    delta_time = param["delta_time"],

                   # memory parameters
                    max_size = param["max_size"],

                   # additional parameters

                    policy = param["policy"],
                    eps = param["eps"],
                    num_episodes = 2,
                    monitoring = True,
                    episode_recording = False,
                    hparams = param.keys())

sumo_RL.train()
# agent.load("./logs/First gridsearch/run_144/model_checkpoints/runFirst gridsearch_iter165000.h5")

# agent.ddqn.train(env = agent.env, num_episodes = 100, policy = agent.policy, connection_label = agent.connection_label)

#evaluation_results = agent.evaluate(runs=2, use_gui= False)
#evaluation_results
# data= agent.ddqn.evaluate(env = agent.env, policy = "greedy")

In [2]:
# GRIDSEARCH
###############################################################

experiment_id = "new_reward2" 


param = { 
    
    "batch_size" : [50], # 30,50,60,[70]
    "target_update_freq" : [20000],  #10000], # 10000,20000,30000,[40000]
    "gamma" : [0.99],# # 0.98,0.99,0.995,[0.999]
    "train_freq" : [1], # 2,3,4,[5]
    "max_size" : [100000],#,70000], # 20000,50000,70000,[100000]
    "max_ep_length" : [2000], # 1000,2000,3000,[4000]
    "policy" : ["epsGreedy", "epsGreedy", "linDecEpsGreedy"],#, "epsGreedy"],
    "eps" : [0.2, 0.5],#, 0.3, 0.1],
    "delta_time" : [10,20],
    "reward" : ["balanced", "negative"]
    #"optimizer": [optimizers.RMSprop(lr= 0.001), optimizers.Adagrad(), optimizers.Adam()]
}

param_grid = iter_params(**param)



def worker(input, output):
    """Runs through a chunk of the grid"""

    for position, args in iter(input.get, 'STOP'):
        result = worker_task(position, args)
        output.put(result)


def worker_task(position, args):
    """Tells the worker what to do with grid chunk"""
    # initialise all obje
    
    # print('Run', position + 1, '-- parameters', args)
    
    sumo_RL = simulation.simulator(
                    connection_label = position + 1,
                    q_network_type = 'simple',
                    target_q_network_type = 'simple',
                    gamma = args["gamma"],
                    target_update_freq = args["target_update_freq"],
                    train_freq = args["train_freq"],
                    num_burn_in = 5000,
                    batch_size = args["batch_size"],
                    optimizer = 'adam',
                    loss_func = "mse",
                    max_ep_length = args["max_ep_length"],
                    experiment_id = experiment_id,
                    model_checkpoint = True,
                    opt_metric = None,

                   # environment parameters
                    net_file = "cross.net.xml",
                    route_file = "cross.rou.xml",
                    demand = "nominal",
                    state_shape = (1,19),
                    num_actions = 2,
                    use_gui = False,
                    delta_time = args["delta_time"],
                    reward = args["reward"],

                   # memory parameters
                    max_size = args["max_size"],

                   # additional parameters

                    policy = args["policy"],
                    eps = args["eps"],
                    num_episodes = 1000,
                    monitoring = True,
                    episode_recording = False,
                    hparams = args.keys())
    
    # print("training agent", position + 1)
    sumo_RL.train()
    # print("evaluating agent", position + 1)
    evaluation_results = sumo_RL.evaluate(runs = 5)

    return (multiprocessing.current_process().name, position + 1, args, evaluation_results["unfinished_runs"],evaluation_results["average_delay"])


def gridsearch(param_grid):
    """Runs a parallelised gridsearch"""

    number_of_processes = multiprocessing.cpu_count()

    # Set up task list
    tasks = [(idx, val) for idx, val in enumerate(param_grid)]

    # Create queues
    task_queue = multiprocessing.Queue()
    done_queue = multiprocessing.Queue()

    # Submit tasks
    for task in tasks:
        task_queue.put(task)
    # Start worker processes
    for i in range(number_of_processes):
        print('Started process #', i + 1)
        multiprocessing.Process(target = worker,
                                args = (task_queue, done_queue)).start()
      
    # Get and print results
    results = []
    for i in range(len(tasks)):
        results.append(done_queue.get())
        with open(os.path.join("./logs",experiment_id,"GS_results"), "a", newline='') as file:
            writer = csv.writer(file, dialect = 'excel')
            writer.writerow(results[-1])                
        print('%s -- [RESULTS]: Run %s -- Parameters %s -- Mean duration %6.0f' % results[-1])
        
    # Tell child processes to stop
    for i in range(number_of_processes):
        task_queue.put('STOP')

    # Now combine the results
    scores = [result[-1] for result in results]
    lowest = min(scores)
    winner = results[scores.index(lowest)]
    return winner, results

multiprocessing.freeze_support()
winner, results = gridsearch(param_grid)
winner

Started process # 1
Started process # 2
Started process # 3
Started process # 4
Started process # 5
Started process # 6
Started process # 7
Started process # 8
...done filling replay memory
Run 5 -- running episode 1 / 600
...done filling replay memory
Run 4 -- running episode 1 / 600
...done filling replay memory
Run 8 -- running episode 1 / 600
...done filling replay memory
Run 2 -- running episode 1 / 600
...done filling replay memory
Run 3 -- running episode 1 / 600
...done filling replay memory
Run 7 -- running episode 1 / 600
...done filling replay memory
Run 1 -- running episode 1 / 600
...done filling replay memory
Run 6 -- running episode 1 / 600
Run 5 -- running episode 2 / 600
Run 6 -- running episode 2 / 600
Run 6 -- running episode 3 / 600
Run 5 -- running episode 3 / 600
Run 3 -- running episode 2 / 600
Run 4 -- running episode 2 / 600
Run 6 -- running episode 4 / 600
Run 5 -- running episode 4 / 600
Run 6 -- running episode 5 / 600
Run 5 -- running episode 5 / 600
Run 6 

Run 6 -- running episode 77 / 600
Run 3 -- running episode 27 / 600
Run 4 -- running episode 28 / 600
Run 5 -- running episode 64 / 600
Run 6 -- running episode 78 / 600
Run 8 -- running episode 11 / 600
Run 2 -- running episode 11 / 600
Run 6 -- running episode 79 / 600
Run 5 -- running episode 65 / 600
Run 7 -- running episode 11 / 600
Run 1 -- running episode 11 / 600
Run 6 -- running episode 80 / 600
Run 6 -- running episode 81 / 600
Run 5 -- running episode 66 / 600
Run 4 -- running episode 29 / 600
Run 3 -- running episode 28 / 600
Run 6 -- running episode 82 / 600
Run 5 -- running episode 67 / 600
Run 6 -- running episode 83 / 600
Run 6 -- running episode 84 / 600
Run 5 -- running episode 68 / 600
Run 6 -- running episode 85 / 600
Run 6 -- running episode 86 / 600
Run 3 -- running episode 29 / 600
Run 4 -- running episode 30 / 600
Run 5 -- running episode 69 / 600
Run 6 -- running episode 87 / 600
Run 6 -- running episode 88 / 600
Run 2 -- running episode 12 / 600
Run 5 -- runni

Run 2 -- running episode 29 / 600
Run 6 -- running episode 163 / 600
Run 4 -- running episode 51 / 600
Run 6 -- running episode 164 / 600
Run 8 -- running episode 19 / 600
Run 5 -- running episode 118 / 600
Run 7 -- running episode 27 / 600
Run 2 -- running episode 30 / 600
Run 6 -- running episode 165 / 600
Run 8 -- running episode 20 / 600
Run 5 -- running episode 119 / 600
Run 6 -- running episode 166 / 600
Run 3 -- running episode 53 / 600
Run 1 -- running episode 19 / 600
Run 6 -- running episode 167 / 600
Run 7 -- running episode 28 / 600
Run 5 -- running episode 120 / 600
Run 2 -- running episode 31 / 600
Run 6 -- running episode 168 / 600
Run 5 -- running episode 121 / 600
Run 4 -- running episode 52 / 600
Run 6 -- running episode 169 / 600
Run 6 -- running episode 170 / 600
Run 7 -- running episode 29 / 600
Run 5 -- running episode 122 / 600
Run 3 -- running episode 54 / 600
Run 2 -- running episode 32 / 600
Run 6 -- running episode 171 / 600
Run 3 -- running episode 55 / 600


Run 7 -- running episode 53 / 600
Run 5 -- running episode 157 / 600
Run 2 -- running episode 56 / 600
Run 6 -- running episode 235 / 600
Run 4 -- running episode 81 / 600
Run 8 -- running episode 33 / 600
Run 6 -- running episode 236 / 600
Run 4 -- running episode 82 / 600
Run 8 -- running episode 34 / 600
Run 5 -- running episode 158 / 600
Run 7 -- running episode 54 / 600
Run 6 -- running episode 237 / 600
Run 4 -- running episode 83 / 600
Run 2 -- running episode 57 / 600
Run 8 -- running episode 35 / 600
Run 3 -- running episode 76 / 600
Run 6 -- running episode 238 / 600
Run 5 -- running episode 159 / 600
Run 8 -- running episode 36 / 600
Run 6 -- running episode 239 / 600
Run 2 -- running episode 58 / 600
Run 8 -- running episode 37 / 600
Run 3 -- running episode 77 / 600
Run 6 -- running episode 240 / 600
Run 5 -- running episode 160 / 600
Run 7 -- running episode 55 / 600
Run 4 -- running episode 84 / 600
Run 6 -- running episode 241 / 600
Run 4 -- running episode 85 / 600
Run

Run 6 -- running episode 292 / 600
Run 7 -- running episode 76 / 600
Run 4 -- running episode 127 / 600
Run 8 -- running episode 64 / 600
Run 6 -- running episode 293 / 600
Run 5 -- running episode 189 / 600
Run 3 -- running episode 98 / 600
Run 2 -- running episode 78 / 600
Run 7 -- running episode 77 / 600
Run 4 -- running episode 128 / 600
Run 8 -- running episode 65 / 600
Run 6 -- running episode 294 / 600
Run 3 -- running episode 99 / 600
Run 5 -- running episode 190 / 600
Run 4 -- running episode 129 / 600
Run 8 -- running episode 66 / 600
Run 6 -- running episode 295 / 600
Run 3 -- running episode 100 / 600
Run 2 -- running episode 79 / 600
Run 4 -- running episode 130 / 600
Run 7 -- running episode 78 / 600
Run 8 -- running episode 67 / 600
Run 6 -- running episode 296 / 600
Run 5 -- running episode 191 / 600
Run 3 -- running episode 101 / 600
Run 4 -- running episode 131 / 600
Run 6 -- running episode 297 / 600
Run 7 -- running episode 79 / 600
Run 3 -- running episode 102 / 6

...

Run 1 -- running episode 290 / 600
Run 16 -- running episode 140 / 600
Run 19 -- running episode 329 / 600
Run 21 -- running episode 119 / 600
Run 18 -- running episode 422 / 600
Run 19 -- running episode 330 / 600
Run 21 -- running episode 120 / 600
Run 23 -- running episode 3 / 600
Run 18 -- running episode 423 / 600
Run 19 -- running episode 331 / 600
Run 21 -- running episode 121 / 600
Run 22 -- running episode 29 / 600
Run 18 -- running episode 424 / 600
Run 19 -- running episode 332 / 600
Run 21 -- running episode 122 / 600
Run 19 -- running episode 333 / 600
Run 18 -- running episode 425 / 600
Run 21 -- running episode 123 / 600
Run 22 -- running episode 30 / 600
Run 20 -- running episode 30 / 600
Run 19 -- running episode 334 / 600
Run 21 -- running episode 124 / 600
Run 18 -- running episode 426 / 600
Run 19 -- running episode 335 / 600
Run 21 -- running episode 125 / 600
Run 18 -- running episode 427 / 600
Run 19 -- running episode 336 / 600
Run 21 -- running episode 126 / 60

Run 19 -- running episode 396 / 600
Run 18 -- running episode 467 / 600
Run 23 -- running episode 15 / 600
Run 21 -- running episode 184 / 600
Run 22 -- running episode 52 / 600
Run 19 -- running episode 397 / 600
Run 18 -- running episode 468 / 600
Run 21 -- running episode 185 / 600
Run 19 -- running episode 398 / 600
Run 18 -- running episode 469 / 600
Run 22 -- running episode 53 / 600
Run 21 -- running episode 186 / 600
Run 23 -- running episode 16 / 600
Run 19 -- running episode 399 / 600
Run 18 -- running episode 470 / 600
Run 21 -- running episode 187 / 600
Run 19 -- running episode 400 / 600
Run 22 -- running episode 54 / 600
Run 21 -- running episode 188 / 600
Run 19 -- running episode 401 / 600
Run 23 -- running episode 17 / 600
Run 1 -- running episode 296 / 600
Run 21 -- running episode 189 / 600
Run 18 -- running episode 471 / 600
Run 19 -- running episode 402 / 600
Run 21 -- running episode 190 / 600
Run 18 -- running episode 472 / 600
Run 22 -- running episode 55 / 600


Run 18 -- running episode 513 / 600
Run 21 -- running episode 243 / 600
Run 20 -- running episode 41 / 600
Run 19 -- running episode 459 / 600
Run 22 -- running episode 79 / 600
Run 18 -- running episode 514 / 600
Run 21 -- running episode 244 / 600
Run 19 -- running episode 460 / 600
Run 21 -- running episode 245 / 600
Run 22 -- running episode 80 / 600
Run 19 -- running episode 461 / 600
Run 21 -- running episode 246 / 600
Run 18 -- running episode 515 / 600
Run 19 -- running episode 462 / 600
Run 21 -- running episode 247 / 600
Run 18 -- running episode 516 / 600
Run 19 -- running episode 463 / 600
Run 21 -- running episode 248 / 600
Run 18 -- running episode 517 / 600
Run 19 -- running episode 464 / 600
Run 22 -- running episode 81 / 600
Run 21 -- running episode 249 / 600
Run 19 -- running episode 465 / 600
Run 23 -- running episode 33 / 600
Run 22 -- running episode 82 / 600
Run 1 -- running episode 302 / 600
Run 21 -- running episode 250 / 600
Run 19 -- running episode 466 / 600

Run 19 -- running episode 530 / 600
Run 22 -- running episode 121 / 600
Run 19 -- running episode 531 / 600
Run 21 -- running episode 288 / 600
Run 22 -- running episode 122 / 600
Run 19 -- running episode 532 / 600
Run 23 -- running episode 40 / 600
Run 22 -- running episode 123 / 600
Run 20 -- running episode 54 / 600
Run 19 -- running episode 533 / 600
Run 1 -- running episode 309 / 600
Run 22 -- running episode 124 / 600
Run 19 -- running episode 534 / 600
Run 22 -- running episode 125 / 600
Run 19 -- running episode 535 / 600
Run 22 -- running episode 126 / 600
Run 19 -- running episode 536 / 600
Run 18 -- running episode 548 / 600
Run 16 -- running episode 159 / 600
Run 19 -- running episode 537 / 600
Run 20 -- running episode 55 / 600
Run 18 -- running episode 549 / 600
Run 19 -- running episode 538 / 600
Run 22 -- running episode 127 / 600
Run 19 -- running episode 539 / 600
Run 20 -- running episode 56 / 600
Run 22 -- running episode 128 / 600
Run 19 -- running episode 540 / 6

Run 18 -- running episode 579 / 600
Run 22 -- running episode 188 / 600
Run 1 -- running episode 319 / 600
Run 20 -- running episode 66 / 600
Run 22 -- running episode 189 / 600
Run 22 -- running episode 190 / 600
Run 18 -- running episode 580 / 600
Run 22 -- running episode 191 / 600
Run 18 -- running episode 581 / 600
Run 23 -- running episode 52 / 600
Run 22 -- running episode 192 / 600
Run 18 -- running episode 582 / 600
Run 22 -- running episode 193 / 600
Run 18 -- running episode 583 / 600
Run 23 -- running episode 53 / 600
Run 16 -- running episode 169 / 600
Run 22 -- running episode 194 / 600
Run 18 -- running episode 584 / 600
Run 22 -- running episode 195 / 600
Run 23 -- running episode 54 / 600
Run 18 -- running episode 585 / 600
Run 22 -- running episode 196 / 600
Run 18 -- running episode 586 / 600
Run 22 -- running episode 197 / 600
Run 23 -- running episode 55 / 600
Run 18 -- running episode 587 / 600
Run 22 -- running episode 198 / 600
Run 21 -- running episode 299 / 60

In [ ]:
winner

In [ ]:
decay_test = []
eps_test = 0.8
for i in range(300000):
    eps_test *= omega_test ** i
    decay_test.append(eps_test)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(np.arange(300000),decay_test)

In [ ]:
first_results = pd.DataFrame(results).sort_values(by = 3)